# Trust in Multi-Agent Systems
Welcome to our lesson on trust and explainability in AI! 

Before we get started, remember to run this cell to get everything set up correctly:

In [6]:
%load_ext autoreload
%autoreload 2
from utils import prep_browser, run_pyperplan, run_and_viz_pyperplan

from pyperplan import task
import re
import mdptoolbox, mdptoolbox.example
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
prep_browser()

<IPython.core.display.Javascript object>

# Explainability
As discussed in lecture, we know that _explaining_ unexpected behaviors from our robot can increase trust. In this module, we'll see how this comes into play. Recall our human-aware planning problem from lecture:

## Human-Aware Planning
Recall the Human-Aware Planning problem described in [1](https://arxiv.org/pdf/2105.01220.pdf).

**Input:**

$\mathcal{M}^R$, the robot's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^R, \mathcal{I}^R, \mathcal{G}^R\rangle$, where $\mathcal{D}^R$ is the domain,  $\mathcal{I}^R$ is the initial state, and $\mathcal{G}^R$ is the goal state.

$\mathcal{M}^G$, the human's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^H, \mathcal{I}^H, \mathcal{G}^H\rangle$, where $\mathcal{D}^H$ is the domain,  $\mathcal{I}^H$ is the initial state, and $\mathcal{G}^H$ is the goal state.

**Output:**

A _plan_; that is, a sequence of robot actions that achieve the goal state but also meets the human's expectations. We call the degree to which the robot plan $\pi$ matches the human expectations $\pi^e$ the plan _explicability_, and we often model it as the _distance_ $\delta$ between $\pi^e$ and $\pi$:

$$
E(\pi) = -1 * \delta(\pi^e, \pi)
$$

A plan $\pi$ is _perfectly explicable_ if $E(\pi) = 0$. We often use the difference in costs between the two plans as our distance function, $\delta$.


## Problem 1: Modeling Incomplete Knowledge
Because we're so familiar with PDDL, we will model both of these problems in PDDL! Check out the references from `ps-03` if you've forgotten how to use it. **TODO: describe this stuff in more detail.**

We'll start with a simplified version of the [Wumpus World](http://users.cecs.anu.edu.au/~patrik/pddlman/wumpus.html). Here, our robot is trying to navigate in a 3x3 grid to pick up a block. The robot knows that its shortest path is unencumbered, but the human has no idea--they think that there's piles of trash on the shortest path to the robot.

![Wumpus World](res/wumpus-world.png)

We've provided you with a fully modeled version of the robot world in `pddl/robot-domain.pddl` and `pddl/robot-problem.pddl`. Your task is to modify `pddl/human-domain.pddl` and `pddl/human-problem.pddl` so it reflects the fact that the _human_ thinks that there is trash in squares `(0, 2)` and `(1, 2)`. Notice that none of our actions are `durative-actions`; we assume a unit cost for every action.

Instead of using Optic to run our plans, we'll use a Python PDDL Planner called [`pyperplan`](https://github.com/aibasel/pyperplan). Pyperplan is cool because it is extensible and contains a very clean implementation of some of the commmon search algorithms; if you're interested in the way planners work, definitely check out their codebase! Like Optic, however, Pyperplan only supports _positive preconditions._  

Note, too, that we cannot specify negative initial conditions. This is important with our `clear` predicate. If we want to say that a square is not `clear` at the start, we simply omit it from the list of clear squares. See `robot-problem.pddl` for a concrete example.

In [8]:
domain_file = 'pddl/robot-domain.pddl'
problem_file = 'pddl/robot-problem.pddl'

run_pyperplan(domain_file, problem_file)

[<Op (robot-move robot1 sq2-2 sq1-2)>, <Op (robot-move robot1 sq1-2 sq0-2)>]

In [9]:
run_and_viz_pyperplan(domain_file, problem_file)

Plan found!

raw planner output:

(robot-move robot1 sq2-2 sq1-2)
(robot-move robot1 sq1-2 sq0-2)


How might we modify this domain to represent the human's belief that there is trash at position $(1, 2)$?

In [10]:
domain_file = 'pddl/human-domain.pddl'
problem_file = 'pddl/human-problem.pddl'

run_and_viz_pyperplan(domain_file, problem_file)

Plan found!

raw planner output:

(robot-move robot1 sq2-2 sq2-1)
(robot-move robot1 sq2-1 sq2-0)
(robot-move robot1 sq2-0 sq1-0)
(robot-move robot1 sq1-0 sq0-0)
(robot-move robot1 sq0-0 sq0-1)
(robot-move robot1 sq0-1 sq0-2)


## Problem 2: Plan Explainer

If you were the human in this human-robot team, you'd probably be concerned if your robot started moving towards where you thought was a gigantic pile of trash! This corresponds to a _decrease in trust_. As discussed in lecture, there are several ways to deal with these trust decreases:

**TODO: enumerate the ways trust can decrease**

Today, we'll be focusing on _explaining_; that is, telling our robot it's probably a good idea to explain its actions when it takes unexpected actions. We'll accomplish this by modifying the PDDL plan with an `explain` action that has some cost.

Let's try writing a function that will perform the robot's optimal plan, but explain to the human why their plans differ. We've implemented a few helper functions below to get you started, as well as outlined some pseudocode in the form of comments in the function `make_explainable_plan.` 

At a high-level, your function should take in as input an optimal robot plan and the expected human plan. It should then compare, action-by-action, the optimal action and the expected action, terminating when the optimal plan has reached the goal state. 

If the actions don't match up, you should check to see if the next optimal action is ever represented in the explainable plan. Then, you should skip to that action in the explainable plan and explain. 

In [11]:
def get_action_agent(operator):
    """
    Returns a string representing the agent completing some action
    Example: get_operation_agent(<Op (robot-move robot1 sq2-0 sq1-0)>)
            should return "robot-1".

    @param operator:    A pyperplan Operator object

    @return:            A string of the agent completing the operation 
    """
    return re.search("(?<=\s)(.*?)(?=\s)", operator.name).group()

def explain(agent):
    """
    Returns an Operator (explain `AGENT`)
    """
    return task.Operator('(explain ' + str(agent) + ')', frozenset(), frozenset(), frozenset())

def calculate_plan_cost(plan):
    """Calculates the cost of a plan. In our case (because each action has a unit cost), 
    the plan cost is simply the length of the plan. """
    return len(plan)

def make_fully_explainable_plan(optimal_plan, expected_plan):
    """
    Makes a fully explainable plan that offers explanations when the expected action 
    differs from the action the robot thinks is best.

    @param optimal_plan:    The list of actions in an optimal plan
    @param expected_plan:   The list of actions in the human-expected plan

    @return explained_plan: The list of actions in the fully explainable plan
    """
    # Initialize explained plan to the empty list
    explained_plan = []

    # Iterate through index in optimal plan
    for i in range(len(optimal_plan)):
        # Get optimal and expected action at step i
        optimal_action = optimal_plan[i]
        expected_action = expected_plan[i]

        # Get the agent performing the optimal action
        agent = get_action_agent(optimal_action)

        # If the optimal action is what you'd expect, 
        # append the optimal action to the explained plan
        if optimal_action == expected_action:
            explained_plan.append(optimal_action)
        # Otherwise, try to find the index "next matching" plan action 
        else:
            try: 
                # See if the optimal action's add effects appear in the expected plan
                # If they do, remove all of the actions in the expected plan
                matching_index = [x.add_effects for x in expected_plan].index(optimal_action.add_effects)
                expected_plan = expected_plan[:i] + expected_plan[matching_index:]
                explained_plan.append(optimal_action)
            except:
                explained_plan.append(optimal_action)
                explained_plan.append(explain(agent))

    # Return the explained plan
    return explained_plan


In [12]:
human_domain_file = 'pddl/human-domain.pddl'
human_problem_file = 'pddl/human-problem.pddl'

robot_domain_file = 'pddl/robot-domain.pddl'
robot_problem_file = 'pddl/robot-problem.pddl'

expected_plan = run_pyperplan(human_domain_file, human_problem_file)
optimal_plan = run_pyperplan(robot_domain_file, robot_problem_file)
explainable_plan = make_fully_explainable_plan(optimal_plan, expected_plan)

# The Meta-MDP

As discussed in class, one way of selecting robot behaviors can be modeled as a "Meta-MDP". We'll use the [Python MDP Toolbox](https://pymdptoolbox.readthedocs.io/en/latest/api/mdp.html). **TODO: Describe the problem, classes, features, etc.**

As in the problem statement, we'll model the problem as an infinite horizon discounted MDP of the form $$M = \langle S, A, P, C, \gamma \rangle$$

### Problem Description
Our **state space**, $S$, are the human's "trust level." For this implementation, we'll have four trust levels, so $\| S \| = 4$. We associate each trust level with numerical values $T = \begin{bmatrix} 0 & 0.3 & 0.6 & 1.0 \end{bmatrix}$, which we will use to help us model the rest of the problem.

Our **action space**, $A$, is simple--the robot may choose between its own _optimal plan_ or the human's fully explainable (or _expected_ plan). (Therefore, $\|A \| = 2$, and $A(0) = \pi^\textrm{opt}$ and $A(1) = \pi^\textrm{exp}$). 

The **explicability score**, $E(\pi)$, is the negative of the cost difference between the current plan and the optimal plan in the robot model. For example, $E(\pi^\textrm{exp}) = - (\textrm{cost}_{\pi^{\textrm{exp}}} - \textrm{cost}_{\pi^{\textrm{opt}}})$.

As described in the [Python MDP Toolbox documentation](https://pymdptoolbox.readthedocs.io/en/latest/api/mdp.html), our transition matrix $P$ should be a [numpy array](https://numpy.org/doc/stable/reference/generated/numpy.array.html) of size `(2, 4, 4)`. Therefore, `P[k][i][j]` represents the likelihood of transitioning from state $s_i$ to state $s_j$ with the action $a_k$. There are two cases to think about when defining our transition matrix:

1. The **optimal plan**. Here, we need to consider three subcases, as our robot is following a plan with a non-perfect explicability score. The trust level may either **decrease**, **stay the same**, or **increase**. 
   - We model the likelihood that the trust level **decreases** as $P(s_i, a^\pi, s_{i-1}) = \omega(i) * (1 - E(\pi))$. 
   - We model the likelihood that the trust level **stays the same** as $P(s_i, a^\pi, s_{i}) = \omega(i) * E(\pi)$.
   - We model the likelihood that the trust level **increases** as $P(s_i, a^\pi, s_{i+1}) = (1 - \omega(i))$
2.  The **expected plan**. Here, trust increases to the next level in all but the maximum trust level (where it is expected to remain the same).

Note that `P(0)` corresponds with the transition matrix of the optimal plan, and `P(1)` corresponds with the transition matrix of the expected plan.


The **cost**, $C$, is modeled as a `numpy array` of size `(4, 2)`. $C(s_i, a^\pi) = (1 - \omega(i)) * C_e(\pi)$, where $C_e(\pi)$ is the cost of the fully explainable plan. Here, we'll assume that each action has unit cost, but this cost function could certainly get more complicated!

The likelihood that the human chooses to observe at some trust level, $\omega(i)$ is modeled as a Bernoulli distribution with probability of $(1 - T(i))$. Here, $\omega$ should be a `numpy array` of size `(1, 4)`.

Your task is to fill in the transition matrix `P` and the reward matrix `R` to model the problem as described above.

In [27]:
# Four trust states
def omega(T):
    """
    Builds the omega matrix. Recall that w(i) = (1 - T(i))

    @param  T:     The numerical values for trust.
    
    @return w:     An np.array of np.shape(T). Recall that w(i)
    """
    
    return np.ones(np.shape(T)) - T

def explicability_score(optimal_cost, expected_cost):
    """
    Builds the explicability score E(pi). Recall that E(plan) = - (plan_cost -
    optimal_plan)

    @param  optimal_cost:   The cost of the optimal plan.
    @param  expected_cost:  The cost of the fully explicable plan.expected_plan

    @return E:             The explicability score given the optimal and explicable plans.
    """
    E = np.array([-(optimal_cost - optimal_cost),
    - (expected_cost - optimal_cost)])
    return E

def transition_matrix(T, w, E):
    """
    Builds the transition matrix, P. Recall that we have 2 actions (size of E)
    and 4 states (size of T). Therefore, our P matrix is an np.array of size
    (length(E), length(T), length(T).


    The first action in P is following the optimal plan. The second action in P
    is following the explainable plan. See the problem statement for a full
    description of our expected transitions.
    
    @param  T: the trust levels
    @param  w: the likelihood that the human observes
    @param  E: the plan explicability score

    @return P: the transition matrix
    """
    P = np.zeros((len(E), len(T), len(T)))

    # following the perfectly explicable plan
    for i in range(len(P[1])):
        P[1][i][min(len(P[0]) - 1, i + 1)] = 1.0

    for i in range(len(P[0])):
        if i != 3:
            P[0][i][i+1] = 1 - w[i] # trust decreases
            P[0][i][i] = w[i] * E[0]
        else:
            P[0][i][i] = 1 - w[i]
        P[0][i][max([0, i-1])] = w[i] * (1 - E[0])

    return P

def cost(w, E, expected_cost):
    C = np.zeros((len(w), len(E)))

    for i in range(len(C)):
        C[i][0] = (1 - w[i]) * expected_cost
        C[i][1] = (1 - w[i]) * expected_cost
    
    return C
        

def build_meta_MDP(T, optimal_plan, expected_plan, gamma):
    """
    Builds the Meta MDP model given trust levels and an optimal and expected plan.

    @param          T: The matrix associating trust level with values in the range [0, 1].
    @param          optimal_plan: A list of actions corresponding with the optimal plan.
    @param          expected_plan: A list of actions corresponding with the explicable/expected plan.

    @return P:      The transition matrix.
    @return C:      The cost matrix.
    @return gamma:  The discount factor
    """

    # Get the w (omega) matrix, representing the likelihood that a human will
    # choose to observe
    w = omega(T)
    
    # Get optimal and expected costs
    optimal_cost = calculate_plan_cost(optimal_plan)
    expected_cost = calculate_plan_cost(expected_plan)

    # Get E (explicability score)
    E = explicability_score(optimal_cost, expected_cost)

    # Get P (transition matrix)
    P = transition_matrix(T, w, E)

    # Get C (cost)
    C = cost(w, E, expected_cost)
        
    return P, C, gamma


Finally, we'll try running the MDP that we wrote! Execute the cell below to see your generated policy.

In [28]:
T = np.array([0, 0.3, 0.6, 1])
P, C, gamma = build_meta_MDP(T, optimal_plan, expected_plan, 0.9)

pi = mdptoolbox.mdp.PolicyIteration(P, C, gamma)
pi.run()

pi.policy

(1, 1, 1, 0)

Notice that your policy takes action $1$ (the explainable plan) in all cases except where human trust is very high. This is what we might expect intuitively, so that's pretty cool!